<a href="https://colab.research.google.com/github/beergil/case-studies/blob/main/artificial_neural_network_binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artifcial Neural Network for Binary Classification

## Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

'2.13.0'

# Part 1 - Data Preprocessing

## Importing the dataset

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [ ]:
X, y

(array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
        [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
        [502, 'France', 'Female', ..., 1, 0, 113931.57],
        ...,
        [709, 'France', 'Female', ..., 0, 1, 42085.58],
        [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
        [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object),
 array([1, 0, 1, ..., 1, 1, 0]))

# Encoding categorical data

### Label Encoding the "Gender" column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


##One Hot Encoding the "Geography" column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(),[1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


## Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.2, random_state=0)

##Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

#Part 2 - Building ANN

## Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()

## Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

## Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

## Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid')) #softmax activation for more than 2 categories

# Part 3 - Training the ANN

## Compling the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss= 'binary_crossentropy', metrics= ['accuracy'])  #categorical_crossentropy for non binary

## Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size= 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.5250 - accuracy: 0.7954
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4573 - accuracy: 0.8002
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4375 - accuracy: 0.8124
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4279 - accuracy: 0.8190
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4206 - accuracy: 0.8269
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4142 - accuracy: 0.8330
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4070 - accuracy: 0.8369
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3995 - accuracy: 0.8404
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3908 - accuracy: 0.8445
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3815 - accura

# Part 4 - Making the predictions and evaluating the model

## Predicting the result of a single observation

### Use our ANN model to predict if the customer with the following informations will leave the bank:

*   Geography: France
*   Credit Score: 600
*   Gender: Male
*   Age: 40 years old
*   Tenure: 3 years
*   Balance: USD 6000
*   Number of Products: 2
*   Does this customer have a credit card: Yes
*   Is this customer an Active Member? Yes
*   Estimated Salary: USD 5000
*   So, should we say goodbye to that customer?


## **Solution**

In [ ]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,5000]])))
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,5000]])) > 0.5)

1/1 [==============================] - 0s 164ms/step
[[0.03169124]]
1/1 [==============================] - 0s 33ms/step
[[False]]


## Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred> 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 4ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


## Making the Confusian Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1529   66]
 [ 206  199]]


0.864